In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

from datetime import datetime

In [2]:
time_start = datetime.now()

vgg16_model_fc1_features = pd.read_csv('vgg16_model_fc1_features.csv', sep=',', header=None)

print('This step took time:', datetime.now() - time_start)

This step took time: 0:00:55.485724


In [3]:
vgg16_model_fc1_features.head()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0.000000,1.474292,0.0,0.0,0.0,2.954327,3.064247,5.079985,0.0,1.183462,...,0.0,0.000000,2.351649,2.044281,0.0,0.0,0.838144,0.0,1.923328,1.126978
1,0.000000,0.000000,0.0,0.0,0.0,1.077510,0.000000,5.343241,0.0,0.000000,...,0.0,0.000000,2.653351,0.642653,0.0,0.0,0.014651,0.0,0.000000,1.543805
2,0.000000,0.000000,0.0,0.0,0.0,0.698330,0.515525,3.865017,0.0,0.000000,...,0.0,0.000000,2.469089,2.300757,0.0,0.0,5.040221,0.0,1.591969,2.947772
3,0.389706,0.000000,0.0,0.0,0.0,2.365584,4.045814,1.669262,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,3.136661,0.0,0.957077,5.003048
4,0.181763,1.061538,0.0,0.0,0.0,4.373861,3.493958,6.155066,0.0,0.000000,...,0.0,0.298311,2.583762,3.086634,0.0,0.0,0.735781,0.0,0.653318,0.000000


In [4]:
vgg16_model_fc1_features.shape

(40000, 4096)

In [5]:
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(vgg16_model_fc1_features.values)

In [8]:
time_start = datetime.now()

kmeans = KMeans(n_clusters=20, max_iter=300, random_state=200)

print('This step took time:', datetime.now() - time_start)

This step took time: 0:00:00


In [9]:
time_start = datetime.now()

kmeans.fit(X)

print('This step took time:', datetime.now() - time_start)

This step took time: 0:13:58.625063


In [ ]:
time_start = datetime.now()

output = kmeans.predict(X)

print('This step took time:', datetime.now() - time_start)

In [ ]:
labels = [i for i in range(20)]
cluster_dict = dict.fromkeys(labels, None) # dict with cluster number as keys and indices of relevant patches as values

for i in range(len(output)):
    if cluster_dict[output[i]] == None:
        cluster_dict[output[i]] = [i]
    else:
        cluster_dict[output[i]].append(i)
        
[len(cluster_dict[key]) for key in cluster_dict]

In [11]:
lfdp_ground_labels = pd.read_csv("LFDP_ground_labels.csv")
lfdp_ground_labels = lfdp_ground_labels.drop('Unnamed: 0', axis=1)

lfdp_palm = lfdp_ground_labels.loc[lfdp_ground_labels['SPECIES'] == 'PREMON']
lfdp_palm = lfdp_palm[lfdp_palm['DIAM'] >= 20]
lfdp_palm

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [34]:
from collections import defaultdict

palm_relevance = defaultdict(int)

for index, entry in lfdp_palm.iterrows():
    patch_y = entry['pix_1']%100
    patch_x = int(entry['pix_2']/100)
    
    patches0_i = patch_x*100 + patch_y
    palm_relevance[output[patches0_i]] += 1
    
for cluster_i in palm_relevance:
    palm_relevance[cluster_i] /= len(cluster_dict[cluster_i])
    
palm_relevance

defaultdict(int,
            {0: 0.0015611340077432247,
             4: 0.0033519553072625698,
             5: 0.0040650406504065045,
             11: 0.0035046728971962616,
             16: 0.0026595744680851063,
             2: 0.00303951367781155,
             10: 0.004690431519699813,
             15: 0.002828854314002829,
             1: 0.0033112582781456954,
             14: 0.0017857142857142857,
             19: 0.001519756838905775})